### Importação das Bibliotecas que serão utilizadas no Modelo

### Aqui segue o pré-processamento e o que foi feito nas Sprints 1 e 2

In [ ]:
!pip install --upgrade tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import gdown
import plotly.express as px
import plotly.graph_objects as go
from tensorflow.keras import backend as K
from sklearn.cluster import KMeans
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download dos arquivos contendo os datasets de consumo desde 2019 a 2024
  - Aqui retiramos a base de 2020, por conta da pandemia do Coronavírus. Foi uma escolha do grupo devido à possibilidade de discrepância nas leituras

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
    "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "CONSUMO_GERAL": "1-IOqfwmh_tTIDHeOer8J-HkGFtwuX67g",
}


dataframes = {}


for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW
From (redirected): https://drive.google.com/uc?id=1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW&confirm=t&uuid=305783c6-72df-404c-8587-31650f0255e2
To: /content/dataset_consumo_2024.csv
100%|██████████| 668M/668M [00:29<00:00, 22.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp
From (redirected): https://drive.google.com/uc?id=1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp&confirm=t&uuid=d8036e86-910e-440c-a189-fb7a058a95ca
To: /content/dataset_consumo_2023.csv
100%|██████████| 1.09G/1.09G [00:25<00:00, 43.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC
From (redirected): https://drive.google.com/uc?id=1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC&confirm=t&uuid=485d5e6f-8aa8-4d6b-ba0a-b05a0cdd0974
To: /content/dataset_consumo_2022.csv
100%|██████████| 1.07G/1.07G [00:13<00:00, 77.0MB/s]
Downloading...
From (original)

In [ ]:
arquivos_csv = [
    "./dataset_consumo_2024.csv",
    "./dataset_consumo_2023.csv",
    "./dataset_consumo_2022.csv",
    "./dataset_consumo_2021.csv",
    "./dataset_consumo_2019.csv",
]

ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)

<ipython-input-5-3dfbd926fda9>:9: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)
<ipython-input-5-3dfbd926fda9>:9: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)
<ipython-input-5-3dfbd926fda9>:9: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)
<ipython-input-5-3dfbd926fda9>:9: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)


In [ ]:
consumo_geral = pd.read_csv('/content/dataset_CONSUMO_GERAL.csv')

## Download do dataset com o Target das Fraudes

In [ ]:
file_id_fraudes = "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
url_fraudes = f"https://drive.google.com/uc?id={file_id_fraudes}"

gdown.download(url_fraudes, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1
To: /content/fraudes.csv
100%|██████████| 55.0M/55.0M [00:01<00:00, 43.7MB/s]


'fraudes.csv'

In [ ]:
fraudes = pd.read_csv('/content/fraudes.csv')

### A tabela "ALL_COLUMNS_CONSUMO_GERAL" possui todas as tabelas de consumo e a partir disso decidimos considerar algumas colunas categóricas que podem ajudar a melhorar o desempenho do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL

,Unnamed: 0,EMP_CODIGO,REFERENCIA,COD_GRUPO,COD_SETOR_COMERCIAL,NUM_QUADRA,COD_ROTA_LEITURA,MATRICULA,SEQ_RESPONSAVEL,ECO_RESIDENCIAL,...,DSC_SIMULTANEA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,COD_LEITURA_INT,STA_TROCA,EXCECAO,STA_ACEITA_LEITURA,COD_LATITUDE,COD_LONGITUDE
0,0,2.0,2024-05-01,4.0,17.0,16.0,11.0,17188010.0,581756.0,0.0,...,63-RETIDA - FATURA ELETRONICA,-21.0,0.0,NaN,901.0,N,Normal,S,-20.460904,-54.634308
1,1,2.0,2024-06-01,4.0,87.0,60.0,38.0,17424465.0,188809.0,1.0,...,00-NAO ENTREGUE,0.0,0.0,NaN,900.0,N,Normal,S,-20.520205,-54.602621
2,2,2.0,2024-07-01,14.0,85.0,314.0,9.0,17857770.0,454315.0,1.0,...,00-NAO ENTREGUE,0.0,0.0,NaN,900.0,N,Normal,S,-20.517992,-54.605950
3,3,2.0,2024-03-01,8.0,5.0,112.0,25.0,17702922.0,311133.0,0.0,...,04-FIXADA AO PORTAO,0.0,0.0,NaN,900.0,N,Normal,S,-20.453648,-54.611187
4,4,2.0,2024-01-01,12.0,88.0,284.0,43.0,17719490.0,735648.0,1.0,...,02-CAIXA CORREIO,-9.0,0.0,NaN,901.0,N,Normal,S,-20.516981,-54.592872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210815,4210815,2.0,2019-11-01,12.0,8.0,271.0,4.0,17108333.0,1049413.0,1.0,...,02-CAIXA CORREIO,10.0,10.0,MEDIA,107.0,N,Normal,N,-20.469254,-54.597804
4210816,4210816,2.0,2019-11-01,9.0,92.0,330.0,47.0,17171758.0,886058.0,1.0,...,04-FIXADA AO PORTAO,10.0,10.0,MEDIA,107.0,N,Normal,N,-20.550235,-54.573375
4210817,4210817,2.0,2019-11-01,11.0,38.0,416.0,20.0,17779236.0,262062.0,1.0,...,02-CAIXA CORREIO,10.0,10.0,MEDIA,104.0,N,Normal,N,-20.438123,-54.666558
4210818,4210818,2.0,2019-04-01,16.0,3.0,76.0,1.0,17338320.0,167787.0,0.0,...,10-ISENTA - NAO IMPRESSA,0.0,0.0,NaN,900.0,N,Normal,S,-20.456004,-54.596311


### Remoção de colunas indesejadas até o momento

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL = ALL_COLUMNS_CONSUMO_GERAL.drop(columns=['Unnamed: 0', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DAT_LEITURA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.columns

Index(['REFERENCIA', 'MATRICULA', 'CONS_MEDIDO', 'TIPO_LIGACAO', 'CATEGORIA',
       'SUB_CATEGORIA', 'DSC_OCORRENCIA', 'VOLUME_ESTIMADO',
       'VOLUME_ESTIMADO_ACUM', 'FATURADO_MEDIA', 'STA_TROCA',
       'STA_ACEITA_LEITURA', 'COD_LATITUDE', 'COD_LONGITUDE'],
      dtype='object')

### Nessa seção queriamos validar a tabela de "VOLUME_ESTIMADO_ACUM" para ver se ela poderia agregar dentro do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[['VOLUME_ESTIMADO_ACUM']].nunique()

,0
VOLUME_ESTIMADO_ACUM,852


In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['VOLUME_ESTIMADO'] != 0]

,REFERENCIA,MATRICULA,CONS_MEDIDO,TIPO_LIGACAO,CATEGORIA,SUB_CATEGORIA,DSC_OCORRENCIA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,STA_TROCA,STA_ACEITA_LEITURA,COD_LATITUDE,COD_LONGITUDE
0,2024-05-01,17188010.0,43.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,-21.0,0.0,NaN,N,S,-20.460904,-54.634308
4,2024-01-01,17719490.0,20.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,NORMAL,-9.0,0.0,NaN,N,S,-20.516981,-54.592872
6,2024-05-01,17839667.0,11.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,-6.0,0.0,NaN,N,S,-20.457056,-54.605360
10,2024-01-01,17369705.0,21.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,NORMAL,-8.0,0.0,NaN,N,S,-20.485598,-54.595796
12,2024-05-01,17270425.0,47.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,NORMAL,-23.0,0.0,NaN,N,S,-20.435660,-54.589481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210813,2019-11-01,17274918.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,MEDIDOR COM MOSTRADOR SUJO,10.0,10.0,MEDIA,N,N,-20.517018,-54.600064
4210814,2019-11-01,17256088.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,MEDIDOR DE DIFÍCIL ACESSO,10.0,10.0,MEDIA,N,N,-20.416534,-54.568428
4210815,2019-11-01,17108333.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,MEDIDOR DE DIFÍCIL ACESSO,10.0,10.0,MEDIA,N,N,-20.469254,-54.597804
4210816,2019-11-01,17171758.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,MEDIDOR DE DIFÍCIL ACESSO,10.0,10.0,MEDIA,N,N,-20.550235,-54.573375


#### O insight retirado aqui é que talvez a melhor coluna para validação e ser utilizada como featura no modelo é a coluna de Volume Estimado. Ela possui maior consistência nos resultados, do que a coluna de Volume Estimado Acumulado

### Separação de Features relevantes

Após pesquisas e visualizar as colunas disponíveis, percebemos uma coluna que poderia ser interessante para o processo de identificação de fraude. A coluna de "DSC_OCORRENCIA". Ela basicamente corresponde a descrição de como foi o processo de vistoria e coleta do responsável e em cada um dos domicílios (matrícula)

Isso surgiu como uma possibilidade de tentar direcionar o modelo para casos nos quais há uma maior possibilidade de uma fraude, de acordo com a visualização do medidor em cada um desses domicílios.

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI = ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['DSC_OCORRENCIA'].isin([
    'NORMAL',
    'MEDIDOR RETIRADO/FURTADO',
    'LEITURA COLETADA PELO CLIENTE',
    'MEDIDOR NÃO LOCALIZADO',
    'IMÓVEL DESOCUPADO'
])]

ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI

,REFERENCIA,MATRICULA,CONS_MEDIDO,TIPO_LIGACAO,CATEGORIA,SUB_CATEGORIA,DSC_OCORRENCIA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,STA_TROCA,STA_ACEITA_LEITURA,COD_LATITUDE,COD_LONGITUDE
0,2024-05-01,17188010.0,43.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,-21.0,0.0,NaN,N,S,-20.460904,-54.634308
1,2024-06-01,17424465.0,0.0,Consumo Fixo,RESIDENCIAL,RESIDENCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.520205,-54.602621
2,2024-07-01,17857770.0,0.0,Consumo Fixo,RESIDENCIAL,RESIDENCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.517992,-54.605950
3,2024-03-01,17702922.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.453648,-54.611187
4,2024-01-01,17719490.0,20.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,NORMAL,-9.0,0.0,NaN,N,S,-20.516981,-54.592872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210780,2019-04-01,17839769.0,0.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.432348,-54.591438
4210781,2019-04-01,17117827.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.500598,-54.642763
4210801,2019-11-01,17086687.0,0.0,Hidrometrado,RESIDENCIAL,RESIDENCIAL,MEDIDOR NÃO LOCALIZADO,18.0,37.0,MEDIA,N,N,-20.468684,-54.625729
4210818,2019-04-01,17338320.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.456004,-54.596311


## Tratando Dataframe com mais colunas targets adicionadas

#### Seperação do Dataframe apenas para a visualização das matriculas com Categoria Pessoa Jurídica

In [ ]:
dataframe_pj_premissa = ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI[ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI["CATEGORIA"].isin(["COMERCIAL", "PUBLICA", "INDUSTRIAL"])]
dataframe_pj_premissa

,REFERENCIA,MATRICULA,CONS_MEDIDO,TIPO_LIGACAO,CATEGORIA,SUB_CATEGORIA,DSC_OCORRENCIA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,STA_TROCA,STA_ACEITA_LEITURA,COD_LATITUDE,COD_LONGITUDE
0,2024-05-01,17188010.0,43.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,-21.0,0.0,NaN,N,S,-20.460904,-54.634308
3,2024-03-01,17702922.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.453648,-54.611187
6,2024-05-01,17839667.0,11.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,-6.0,0.0,NaN,N,S,-20.457056,-54.605360
9,2024-03-01,17106269.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.475057,-54.607668
14,2024-07-01,17269218.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.421107,-54.577934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210778,2019-04-01,17839086.0,0.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.454158,-54.615851
4210779,2019-04-01,17838898.0,0.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.482753,-54.640831
4210780,2019-04-01,17839769.0,0.0,Hidrometrado,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.432348,-54.591438
4210818,2019-04-01,17338320.0,0.0,Consumo Fixo,COMERCIAL,COMERCIAL,NORMAL,0.0,0.0,NaN,N,S,-20.456004,-54.596311


### Processo de One Hot Encoding para as Colunas, as quais serão as features para o nosso modelo

In [ ]:
dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns=['TIPO_LIGACAO', 'DSC_OCORRENCIA', 'STA_TROCA', 'STA_ACEITA_LEITURA'], dtype='int')

In [ ]:
dataframe_pj_premissa

,REFERENCIA,MATRICULA,CONS_MEDIDO,CATEGORIA,SUB_CATEGORIA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,COD_LATITUDE,COD_LONGITUDE,...,TIPO_LIGACAO_Hidrometrado,DSC_OCORRENCIA_IMÓVEL DESOCUPADO,DSC_OCORRENCIA_LEITURA COLETADA PELO CLIENTE,DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO,DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO,DSC_OCORRENCIA_NORMAL,STA_TROCA_N,STA_TROCA_S,STA_ACEITA_LEITURA_N,STA_ACEITA_LEITURA_S
0,2024-05-01,17188010.0,43.0,COMERCIAL,COMERCIAL,-21.0,0.0,NaN,-20.460904,-54.634308,...,1,0,0,0,0,1,1,0,0,1
3,2024-03-01,17702922.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.453648,-54.611187,...,0,0,0,0,0,1,1,0,0,1
6,2024-05-01,17839667.0,11.0,COMERCIAL,COMERCIAL,-6.0,0.0,NaN,-20.457056,-54.605360,...,1,0,0,0,0,1,1,0,0,1
9,2024-03-01,17106269.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.475057,-54.607668,...,0,0,0,0,0,1,1,0,0,1
14,2024-07-01,17269218.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.421107,-54.577934,...,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210778,2019-04-01,17839086.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.454158,-54.615851,...,1,0,0,0,0,1,1,0,0,1
4210779,2019-04-01,17838898.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.482753,-54.640831,...,1,0,0,0,0,1,1,0,0,1
4210780,2019-04-01,17839769.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.432348,-54.591438,...,1,0,0,0,0,1,1,0,0,1
4210818,2019-04-01,17338320.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.456004,-54.596311,...,0,0,0,0,0,1,1,0,0,1


In [ ]:
fraudes.columns

Index(['TIPOOS', 'ANOOS', 'IDOSP', 'ANOMES', 'MATRICULA', 'OS', 'SERVICO',
       'DESCRICAO', 'COD_GRUPO', 'SETOR', 'DATACONCLUSAO', 'STATUS', 'ATRASO',
       'DESCRSETOR', 'DESCRSETORSOLICITANTE', 'ID_CONTRATO', 'CD_CONTRATO',
       'CD_SUB_REGIAO', 'CD_REGIAO', 'FL_PRINCIPAL', 'DS_SERVICO_SOLICITADO',
       'FL_EXECUTADO', 'DT_SERVICO', 'DT_FECHAMENTO', 'PARECER_EXECUCAO'],
      dtype='object')

In [ ]:
fraudes

,TIPOOS,ANOOS,IDOSP,ANOMES,MATRICULA,OS,SERVICO,DESCRICAO,COD_GRUPO,SETOR,...,ID_CONTRATO,CD_CONTRATO,CD_SUB_REGIAO,CD_REGIAO,FL_PRINCIPAL,DS_SERVICO_SOLICITADO,FL_EXECUTADO,DT_SERVICO,DT_FECHAMENTO,PARECER_EXECUCAO
0,Desdobro,2023,230101031796,11/23,17229588,1031796,110013,IRREGULARIDADE IDENTIFICADA,0.0,80.0,...,41,41,0.0,0.0,0,VISTORIA DE IRREGULARIDADE - IMPEDIMENTO DE AC...,1,2023-11-06 0:00:00,2023-11-06 11:07:02,NaN
1,Desdobro,2024,240100141765,02/24,17804014,141765,110013,IRREGULARIDADE IDENTIFICADA,0.0,70.0,...,41,41,0.0,0.0,0,VISTORIA DE IRREGULARIDADE,1,2024-02-13 0:00:00,2024-02-13 15:59:01,421.0
2,Desdobro,2024,240100021314,01/24,17234771,21314,110013,IRREGULARIDADE IDENTIFICADA,0.0,96.0,...,41,41,0.0,0.0,0,VISTORIA DE IRREGULARIDADE IDENTIFICADA - LEITURA,1,2024-01-08 0:00:00,2024-01-08 15:45:46,413.0
3,Desdobro,2023,230101217142,12/23,17837656,1217142,110013,IRREGULARIDADE IDENTIFICADA,0.0,1.0,...,41,41,0.0,0.0,0,VISTORIA DE IRREGULARIDADE SUSPEITA - LEITURISTA,1,2024-01-03 0:00:00,2024-01-03 15:40:12,NaN
4,Desdobro,2024,240100077627,01/24,17722316,77627,110013,IRREGULARIDADE IDENTIFICADA,0.0,84.0,...,41,41,0.0,0.0,0,VISTORIA DE IRREGULARIDADE - IMPEDIMENTO DE AC...,1,2024-01-24 0:00:00,2024-01-24 08:53:18,421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225992,OsOnline,2020,200100034631,01/20,17865556,100034631,2039,IRREGULARIDADE IDENTIFICADA,4.0,92.0,...,53,18,1.0,1.0,0,VERIFICACAO DE IRREGULARIDADE,1,2020-01-14 14:00:07,2020-01-14 14:42:56,NaN
225993,OsOnline,2022,220100065876,01/22,17511826,100065876,110013,IRREGULARIDADE IDENTIFICADA,4.0,76.0,...,53,18,1.0,1.0,0,VISTORIA DE IRREGULARIDADE - DENUNCIA,1,2022-01-21 10:38:13,2022-01-21 10:38:23,NaN
225994,OsOnline,2022,220100450752,05/22,17920302,100450752,110013,IRREGULARIDADE IDENTIFICADA,4.0,3.0,...,53,18,2.0,1.0,0,VISTORIA DE IRREGULARIDADE IDENTIFICADA - LEITURA,1,2022-05-10 16:51:43,2022-05-10 16:52:03,NaN
225995,OsOnline,2022,220101353952,12/22,17801545,101353952,110013,IRREGULARIDADE IDENTIFICADA,4.0,2.0,...,53,18,2.0,1.0,0,VISTORIA POS CORTE,1,2022-12-30 11:52:39,2022-12-30 12:22:23,NaN


In [ ]:
dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
dataframe_fraudes_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].unique()

array([1])

In [ ]:
len(dataframe_fraudes_premissa)

124168

In [ ]:
dataframe_pj_premissa = pd.merge(dataframe_pj_premissa, dataframe_fraudes_premissa, on='MATRICULA', how='left')

In [ ]:
# dataframe_pj_premissa.drop_duplicates(subset="MATRICULA", keep='first')
dataframe_pj_premissa.dropna(subset="REFERENCIA")

,REFERENCIA,MATRICULA,CONS_MEDIDO,CATEGORIA,SUB_CATEGORIA,VOLUME_ESTIMADO,VOLUME_ESTIMADO_ACUM,FATURADO_MEDIA,COD_LATITUDE,COD_LONGITUDE,...,DSC_OCORRENCIA_IMÓVEL DESOCUPADO,DSC_OCORRENCIA_LEITURA COLETADA PELO CLIENTE,DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO,DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO,DSC_OCORRENCIA_NORMAL,STA_TROCA_N,STA_TROCA_S,STA_ACEITA_LEITURA_N,STA_ACEITA_LEITURA_S,DESCRICAO_IRREGULARIDADE IDENTIFICADA
0,2024-05-01,17188010.0,43.0,COMERCIAL,COMERCIAL,-21.0,0.0,NaN,-20.460904,-54.634308,...,0,0,0,0,1,1,0,0,1,NaN
1,2024-03-01,17702922.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.453648,-54.611187,...,0,0,0,0,1,1,0,0,1,NaN
2,2024-05-01,17839667.0,11.0,COMERCIAL,COMERCIAL,-6.0,0.0,NaN,-20.457056,-54.605360,...,0,0,0,0,1,1,0,0,1,NaN
3,2024-03-01,17106269.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.475057,-54.607668,...,0,0,0,0,1,1,0,0,1,NaN
4,2024-07-01,17269218.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.421107,-54.577934,...,0,0,0,0,1,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742512,2019-04-01,17839086.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.454158,-54.615851,...,0,0,0,0,1,1,0,0,1,NaN
1742513,2019-04-01,17838898.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.482753,-54.640831,...,0,0,0,0,1,1,0,0,1,1.0
1742514,2019-04-01,17839769.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.432348,-54.591438,...,0,0,0,0,1,1,0,0,1,NaN
1742515,2019-04-01,17338320.0,0.0,COMERCIAL,COMERCIAL,0.0,0.0,NaN,-20.456004,-54.596311,...,0,0,0,0,1,1,0,0,1,NaN


In [ ]:
dataframe_pj_premissa = dataframe_pj_premissa.dropna(subset="COD_LATITUDE")
dataframe_pj_premissa = dataframe_pj_premissa.dropna(subset="COD_LONGITUDE")

## Clusterização

### Agrupamento dos clusteres por Latitude e Longitude

Aqui removendo os outliers nas colunas de latitude e longitude

In [ ]:
def remove_rows_with_column_value_greater_than_one(df, column_names):
    condition = (df[column_names] > -10).any(axis=1)

    df_filtered = df[~condition]

    return df_filtered

dataframe_pj_premissa = remove_rows_with_column_value_greater_than_one(dataframe_pj_premissa, ['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
def plotar(n_clusters):

  df_temp = dataframe_pj_premissa.copy()
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  df_temp['cluster'] = kmeans.fit_predict(dataframe_pj_premissa[['COD_LATITUDE', 'COD_LONGITUDE']])

  # visualizando
  plt.figure(figsize=(10, 6))
  plt.scatter(dataframe_pj_premissa['COD_LONGITUDE'], dataframe_pj_premissa['COD_LATITUDE'], c=df_temp['cluster'], cmap='viridis', marker='o', s=100)
  plt.title(f"K-Means Clustering com {n_clusters} Clusters")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  plt.show()


# visulizar clusters do range de 2 para 17
# for x in range(2, 17):
#   print()
#   plotar(x)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
dataframe_pj_premissa['cluster'] = kmeans.fit_predict(dataframe_pj_premissa[['COD_LATITUDE', 'COD_LONGITUDE']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
dataframe_pj_premissa = dataframe_pj_premissa.drop(columns=['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns = ['cluster'], dtype=int)

### Normalizando com o Robust Scaler

O RobustScaler é uma técnica de normalização usada para transformar dados. Ele é útil especialmente quando os dados contêm outliers, ou seja, valores atípicos que podem distorcer (neste caso o Consumo e o Volume) os resultados de outras técnicas de escalonamento, como StandardScaler ou MinMaxScaler.

O RobustScaler transforma os dados subtraindo a mediana e dividindo pela amplitude interquartil (IQR, Interquartile Range). A mediana é o valor do ponto médio quando os dados são ordenados, e o IQR é a diferença entre o terceiro quartil (75º percentil) e o primeiro quartil (25º percentil).

Esse método é menos sensível a outliers porque, ao contrário da média e do desvio padrão (usados pelo StandardScaler), a mediana e o IQR não são afetados por valores extremos.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

dataframe_pj_premissa[['CONS_MEDIDO']] = scaler.fit_transform(dataframe_pj_premissa[['CONS_MEDIDO']])
dataframe_pj_premissa[['VOLUME_ESTIMADO']] = scaler.fit_transform(dataframe_pj_premissa[['VOLUME_ESTIMADO']])

In [ ]:
pivoted_df = pd.pivot_table(
    dataframe_pj_premissa,
    index='MATRICULA',
    columns='REFERENCIA',
    values=['CONS_MEDIDO', 'VOLUME_ESTIMADO'],
    aggfunc='sum'
)

pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
pivoted_df = pivoted_df.reset_index()

pivoted_df.head()

,MATRICULA,CONS_MEDIDO_2019-01-01,CONS_MEDIDO_2019-02-01,CONS_MEDIDO_2019-03-01,CONS_MEDIDO_2019-04-01,CONS_MEDIDO_2019-05-01,CONS_MEDIDO_2019-06-01,CONS_MEDIDO_2019-07-01,CONS_MEDIDO_2019-08-01,CONS_MEDIDO_2019-09-01,...,VOLUME_ESTIMADO_2023-11-01,VOLUME_ESTIMADO_2023-12-01,VOLUME_ESTIMADO_2024-01-01,VOLUME_ESTIMADO_2024-02-01,VOLUME_ESTIMADO_2024-03-01,VOLUME_ESTIMADO_2024-04-01,VOLUME_ESTIMADO_2024-05-01,VOLUME_ESTIMADO_2024-06-01,VOLUME_ESTIMADO_2024-07-01,VOLUME_ESTIMADO_2024-08-01
0,17075331.0,0.428571,0.571429,0.571429,NaN,1.714286,0.714286,0.714286,0.857143,0.857143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17075333.0,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17075335.0,-0.142857,0.000000,-0.142857,-0.142857,-0.142857,-0.428571,-0.285714,0.000000,1.285714,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17075337.0,-0.285714,-0.285714,-0.285714,0.285714,-0.142857,-0.142857,0.000000,-0.142857,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17075338.0,1.000000,0.571429,1.142857,1.285714,0.714286,1.000000,0.857143,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
len(pivoted_df)

46499

### Atribuição de cada uma das variáveis categóricas ao dataframe com o consumo e volume históricos

- É importante citar aqui que atribuímos a premissa que a primeira definição de tipo de ligação, descrição e os outros utilizados, serão os que tomaremos como base para inferência dentro do modelo

In [ ]:
tipo_ligacao = dataframe_pj_premissa[['MATRICULA','TIPO_LIGACAO_Consumo Fixo', 'TIPO_LIGACAO_Hidrometrado']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(tipo_ligacao, on='MATRICULA', how='left').fillna(0)

In [ ]:
descricao_ocorrencia = dataframe_pj_premissa[['MATRICULA','DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO', 'DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO', 'DSC_OCORRENCIA_NORMAL']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(descricao_ocorrencia, on='MATRICULA', how='left').fillna(0)

In [ ]:
fraude_ou_não = dataframe_pj_premissa[['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(fraude_ou_não, on='MATRICULA', how='left').fillna(0)

In [ ]:
dataframe_pj_premissa.columns.values

array(['REFERENCIA', 'MATRICULA', 'CONS_MEDIDO', 'CATEGORIA',
       'SUB_CATEGORIA', 'VOLUME_ESTIMADO', 'VOLUME_ESTIMADO_ACUM',
       'FATURADO_MEDIA', 'TIPO_LIGACAO_Consumo Fixo',
       'TIPO_LIGACAO_Hidrometrado', 'DSC_OCORRENCIA_IMÓVEL DESOCUPADO',
       'DSC_OCORRENCIA_LEITURA COLETADA PELO CLIENTE',
       'DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO',
       'DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO', 'DSC_OCORRENCIA_NORMAL',
       'STA_TROCA_N', 'STA_TROCA_S', 'STA_ACEITA_LEITURA_N',
       'STA_ACEITA_LEITURA_S', 'DESCRICAO_IRREGULARIDADE IDENTIFICADA',
       'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13',
       'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17',
       'cluster_18', 'cluster_19'], dtype=object)

In [ ]:
clusters = dataframe_pj_premissa[['MATRICULA','cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13',
       'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17',
       'cluster_18', 'cluster_19']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(clusters, on='MATRICULA', how='left').fillna(0)

In [ ]:
pivoted_df = pivoted_df.fillna(0)

In [ ]:
pivoted_df

,MATRICULA,CONS_MEDIDO_2019-01-01,CONS_MEDIDO_2019-02-01,CONS_MEDIDO_2019-03-01,CONS_MEDIDO_2019-04-01,CONS_MEDIDO_2019-05-01,CONS_MEDIDO_2019-06-01,CONS_MEDIDO_2019-07-01,CONS_MEDIDO_2019-08-01,CONS_MEDIDO_2019-09-01,...,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cluster_18,cluster_19
0,17075331.0,0.428571,0.571429,0.571429,0.000000,1.714286,0.714286,0.714286,0.857143,0.857143,...,0,0,0,0,0,0,1,0,0,0
1,17075333.0,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,-0.571429,...,0,0,0,0,0,0,1,0,0,0
2,17075335.0,-0.142857,0.000000,-0.142857,-0.142857,-0.142857,-0.428571,-0.285714,0.000000,1.285714,...,0,0,0,0,0,0,1,0,0,0
3,17075337.0,-0.285714,-0.285714,-0.285714,0.285714,-0.142857,-0.142857,0.000000,-0.142857,0.000000,...,0,0,0,0,0,0,1,0,0,0
4,17075338.0,1.000000,0.571429,1.142857,1.285714,0.714286,1.000000,0.857143,1.000000,1.000000,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46494,17997580.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
46495,17997582.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
46496,17997584.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
46497,17997585.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
np.unique(pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

(array([0., 1.]), array([32453, 14046]))

In [ ]:
pivoted_df.to_parquet('/content/drive/Shareddrives/LeakSeeker/pre_processado_pj.parquet')